# Facility Port Example

In [ ]:
import sys
import os
import json
import traceback
import time
from datetime import datetime, timedelta
from dateutil import tz
from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network

## Chameleon Imports

In [ ]:
import chi
import chi.lease

## Fabric Imports

In [ ]:
from fabrictestbed_extensions.fablib.fablib import fablib

## Chameleon Variables

For stitching to work we need to specify stich provider "fabric" and physical network "physnet1"

In [ ]:
# Chameleon Config
chameleon_prefix =  f"chameleon_fabric_facility_port_"
chameleon_server_name = chameleon_prefix+'server'
chameleon_network_name = chameleon_prefix+'net'
chameleon_subnet_name = chameleon_prefix+'subnet'
chameleon_router_name = chameleon_prefix+'router'
chameleon_lease_name = chameleon_prefix+'lease'

chameleon_image_name='CC-Ubuntu20.04'
chameleon_node_type="compute_cascadelake_r"
chameleon_physical_network='physnet1'
chameleon_stitch_provider='fabric'
chameleon_server_count=2
chameleon_key_name='fabric-chameleon'

## Fabric Variables

In [ ]:
# Create a FABlib manager
fablib.show_config()

# FABRIC Config
fabric_slice_name=f'chameleon_fabric_facility_port'
fabric_node_name='node1'

fabric_node_image='default_ubuntu_20'
fabric_site='STAR'
fabric_cores = 24
fabric_ram = 32
fabric_disk = 100

## Configure Network Space

In [ ]:
#Network Config
subnet = IPv4Network("192.168.100.0/24")

fabric_allocation_pool_start=IPv4Address('192.168.100.200')
fabric_allocation_pool_end=IPv4Address('192.168.100.250')
fabric_available_ips=[]
for ip_int in range(int(fabric_allocation_pool_start),int(fabric_allocation_pool_end)+1):
    fabric_available_ips.append(IPv4Address(ip_int))
    
chameleon_allocation_pool_start='192.168.100.100'
chameleon_allocation_pool_end='192.168.100.150'
chameleon_gateway_ip='192.168.100.1'

#chameleon_available_ips=[]
#for ip_int in range(int(IPv4Address(chameleon_allocation_pool_start)),int(IPv4Address(chameleon_allocation_pool_end))+1):
#    chameleon_available_ips.append(IPv4Address(ip_int))

## Create Chameleon Lease For Compute and Network

In [ ]:
BLAZAR_TIME_FORMAT = '%Y-%m-%d %H:%M'

try:
    # Set start/end date for lease
    # Start one minute into future to avoid Blazar thinking lease is in past
    # due to rounding to closest minute.
    start_date = (datetime.now(tz=tz.tzutc()) + timedelta(minutes=1)).strftime(BLAZAR_TIME_FORMAT)
    end_date   = (datetime.now(tz=tz.tzutc()) + timedelta(days=1)).strftime(BLAZAR_TIME_FORMAT)

    # Build list of reservations (in this case there is only one reservation)
    reservation_list = []
    chi.lease.add_node_reservation(reservation_list, count=chameleon_server_count, node_type=chameleon_node_type)

    reservation_list.append(
            {
                "resource_type": "network",
                "network_name": chameleon_network_name,
                "network_properties": "",
                "resource_properties": json.dumps(
                    ["==", "$stitch_provider", chameleon_stitch_provider]
                ),
            }
    )
    
    #reservation_list.append(
    #        {
    #            "resource_type": "network",
    #            "network_name": chameleon_network_name,
    #            "network_properties": "",
    #            "resource_properties": json.dumps(
    #                ["==", "$segment_id", "3303"]
    #            ),
    #        }
    #)


    # Create the lease
    chameleon_lease = chi.lease.create_lease(chameleon_lease_name,
                                      reservations=reservation_list,
                                      start_date=start_date,
                                      end_date=end_date)

    #Print the lease info
    chameleon_compute_reservation_id = [reservation for reservation in chameleon_lease['reservations'] if reservation['resource_type'] == 'physical:host'][0]['id']
    print(f"chameleon_compute_reservation_id: {chameleon_compute_reservation_id}")
    chameleon_network_reservation_id = [reservation for reservation in chameleon_lease['reservations'] if reservation['resource_type'] == 'network'][0]['id']
    print(f"chameleon_network_reservation_id: {chameleon_network_reservation_id}")
except Exception as e:
    print(f"Exception: {e}")
    traceback.print_exc()

## Wait Until Network Gets Provisioned

In [ ]:
chameleon_network = None
chameleon_network_id = None
network_vlan = None

while network_vlan == None:
    try:
        #Get the network
        chameleon_network = chi.network.get_network(chameleon_network_name)

        #Get the network ID
        chameleon_network_id = chameleon_network['id']
        print(f'Chameleon Network ID: {chameleon_network_id}')

        #Get the VLAN tag (needed for FABRIC stitching)
        network_vlan = chameleon_network['provider:segmentation_id']
        print(f'network_vlan: {network_vlan}')
    except:
        print(f'Chameleon Network is not ready. Trying again!')
        time.sleep(10)           

## Create The Subnet

In [ ]:
try:
    chameleon_subnet = chi.network.create_subnet(chameleon_subnet_name, chameleon_network_id, 
                                             cidr=str(subnet),
                                             allocation_pool_start=chameleon_allocation_pool_start,
                                             allocation_pool_end=chameleon_allocation_pool_end,
                                             gateway_ip=chameleon_gateway_ip)
    chameleon_subnet_id = chameleon_subnet["id"]
except Exception as e:
    print(f"Exception: {e}")
    traceback.print_exc()

print(json.dumps(chameleon_subnet, indent=2))

## Create The Router and Add The Subnet

In [ ]:
try:
    chameleon_router = chi.network.create_router(chameleon_router_name, gw_network_name='public')
    chi.network.add_subnet_to_router_by_name(chameleon_router_name, chameleon_subnet_name)
except Exception as e:
    print(f"Exception: {e}")
    traceback.print_exc()
    
print(json.dumps(chameleon_router, indent=2))

## Create Fabric Slice With Facility Port

Facility ports currently only work with the dedicate Smart Nics (ConnectX_5 and ConnectX_6)
Additionally stitching can work from within the same site that has the Facility Port or from VMs in different sites, but the connection type is L2PTP.
Finally each Facility Port has its own permission on Fabric which the project needs to gain access before using

In [ ]:
try:
    #Create a slice
    fabric_slice = fablib.new_slice(name=fabric_slice_name)
    
    fabric_node = fabric_slice.add_node(name=fabric_node_name, site=fabric_site, image=fabric_node_image,
                                       cores=fabric_cores, ram=fabric_ram, disk=fabric_disk)

    fabric_node_iface = fabric_node.add_component(model='NIC_ConnectX_5', name=f"stitch_nic").get_interfaces()[0]
    
    fabric_facility_port = fabric_slice.add_facility_port(name='Chameleon-StarLight', site='STAR', vlan=str(network_vlan))
    fabric_facility_port_iface = fabric_facility_port.get_interfaces()[0]
    
    fabric_net = fabric_slice.add_l2network(name=f'net_facility_port', interfaces=[fabric_node_iface,fabric_facility_port_iface]) 

    #Submit the Request
    fabric_slice.submit()
except Exception as e:
    print(f"Exception: {e}")
    traceback.print_exc()

## Set IP address on the Fabric Node Connected to the Facility Port

In [ ]:
try:        
    fabric_node = fabric_slice.get_node(name=fabric_node_name)   
    print(f"{fabric_node}")
    
    fabric_node_iface = fabric_node.get_interface(network_name=f'net_facility_port') 
    fabric_node_addr = fabric_available_ips.pop(0)
    print(f"fabric_node_addr: {fabric_node_addr}")
    fabric_node_iface.ip_addr_add(addr=fabric_node_addr, subnet=subnet)
    
    stdout, stderr = fabric_node.execute(f'ip addr show {fabric_node_iface.get_os_interface()}')
    print (stdout)    
except Exception as e:
    print(f"Exception: {e}")

## Ping Chameleon Router From Fabric

In [ ]:
try:
    fabric_node = fabric_slice.get_node(name=fabric_node_name) 
    stdout, stderr = fabric_node.execute(f'ping -c 4 {chameleon_gateway_ip}')
    print (stdout)
except Exception as e:
    print(f"Error: {e}")

## Create Chameleon Nodes If Router Responds to Ping

In [ ]:
servers=[]

try:
    for i in range(chameleon_server_count):
        server_name=f"{chameleon_server_name}_{i}"
        port_name=f"{chameleon_server_name}_port_{i}"
        
        # Create the server
        servers.append(chi.server.create_server(server_name, 
                              reservation_id=chameleon_compute_reservation_id, 
                              network_name=chameleon_network_name, 
                              image_name=chameleon_image_name,
                              key_name=chameleon_key_name))

except Exception as e:
    print(f"Exception: {e}")

for server in servers:
    chi.server.wait_for_active(server.id)

## Get Fixed IPs

In [ ]:
fixed_ips={}
for i in range(chameleon_server_count):
    server_name=f"{chameleon_server_name}_{i}"
    server_id = get_server_id(server_name)
    fixed_ip = get_server(server_id).interface_list()[0].to_dict()["fixed_ips"][0]["ip_address"]
    fixed_ips[server_name]=fixed_ip

for server_name,fixed_ip in fixed_ips.items():
    print(f'{server_name}: {fixed_ip}')

## Ping Chameleon Nodes From Fabric

In [ ]:
try:
    fabric_node = fabric_slice.get_node(name=fabric_node_name) 
    stdout, stderr = fabric_node.execute(f'ping -c 4 192.168.100.135')
    print (stdout)
except Exception as e:
    print(f"Error: {e}")

## Cleanup Chameleon

In [ ]:
for i in range(chameleon_server_count):
    server_name=f"{chameleon_server_name}_{i}"
    chi.server.delete_server(chi.server.get_server_id(server_name))

In [ ]:
router_id = chameleon_router['id']
subnet_id = chameleon_subnet['id']
network_id = chameleon_network_id

try:
    result = chi.network.remove_subnet_from_router(router_id, subnet_id)
except Exception as e:
    print(f"detach_router error: {str(e)}")

In [ ]:
try:
    result = chi.network.delete_router(router_id)
except Exception as e:
    print(f"delete_router error: {str(e)}")

In [ ]:
try:
    result = chi.network.delete_subnet(subnet_id)
except Exception as e:
    print(f"delete_subnet error: {str(e)}")

In [ ]:
try:
    result = chi.network.delete_network(network_id)
except Exception as e:
    print(f"delete_network error: {str(e)}")

In [ ]:
try:
    chi.lease.delete_lease(chameleon_lease['id'])
except Exception as e:
    print(f"delete_lease error: {str(e)}")

## Cleanup Fabric

In [ ]:
try:
    fabric_slice = fablib.get_slice(fabric_slice_name)
    fabric_slice.delete()
except Exception as e:
    print(f"Exception: {e}")